In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/re_decisioning
/Users/shashankgupta/Documents/code/git_project/re_decisioning/Code


In [2]:
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
import pandas as pd
import numpy as np
import chart_studio.plotly as py 
from datetime import timedelta
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from pandas_profiling import ProfileReport
pio.templates.default = 'plotly_dark'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_map = {'high': '#ef553b', 'low': '#636efa'}
import datetime
import pytz

In [3]:
querySno = SQLQuery('snowflake')
pilot_df = querySno(r"""
with cashins as (
select product_id, business_id,
max(case when rnk = 1 then transaction_id else null end) as first_cashin_trans_id,
max(case when rnk = 2 then transaction_id else null end) as second_cashin_trans_id,
max(case when rnk = 3 then transaction_id else null end) as third_cashin_trans_id,
max(case when rnk = 1 then a.created_at else null end) as first_cashin_date,
max(case when rnk = 2 then a.created_at else null end) as second_cashin_date,
max(case when rnk = 3 then a.created_at else null end) as third_cashin_date
--min(transaction_date) as first_squareoff_date

from (
select * from 
(select *, row_number() over(partition by product_id order by created_at) rnk
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LOAN_PRODUCT_TRANSACTIONS
where action_type='CASHIN' 
and (method = 'AUTOMATED' or description ilike 'Cashin done by admin%%')
--and created_at < to_date('2022-12-01')
)
where rnk<4
) a 
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LOAN_PRODUCTS b on a.product_id = b.id

group by 1,2
),

first_squareoff as(
select product_id,transaction_date as first_squareoff_date from (
select a.*,
f.transaction_date, row_number() over(partition by a.business_id order by f.transaction_date) rnk
from cashins a
left join prod_db.data.transactions f on a.business_id = f.business_id
where date(f.transaction_date) >= date(a.first_cashin_date)
and f.running_balance >= 0
)
where rnk = 1
),

second_squareoff as(
select product_id,transaction_date as second_squareoff_date from (
select a.*,
s.transaction_date, row_number() over(partition by a.business_id order by s.transaction_date) rnk
from cashins a
left join prod_db.data.transactions s on a.business_id = s.business_id
where date(s.transaction_date) >= date(a.second_cashin_date)
and s.running_balance >= 0
)
where rnk = 1
),

third_squareoff as(
select product_id,transaction_date as third_squareoff_date from (
select a.*,
t.transaction_date, row_number() over(partition by a.business_id order by t.transaction_date) rnk
from cashins a
left join prod_db.data.transactions t on a.business_id = t.business_id
where date(t.transaction_date) >= date(a.third_cashin_date)
and t.running_balance >= 0
)
where rnk = 1
)

select a.product_id, a.business_id,first_cashin_date as pilot_first_due_date, second_cashin_date as pilot_second_due_date,third_cashin_date as pilot_third_due_date,
case when f.running_balance < 0 then 1 else 0 end as pilot_fpd,
case when s.running_balance < 0 then 1 else 0 end as pilot_spd,
case when t.running_balance < 0 then 1 else 0 end as pilot_tpd,
datediff(day,a.first_cashin_date,fs.first_squareoff_date) as pilot_first_cure_days,
datediff(day,a.second_cashin_date,ss.second_squareoff_date) as pilot_second_cure_days,
datediff(day,a.third_cashin_date,ts.third_squareoff_date) as pilot_third_cure_days

from "PROD_DB"."ADHOC"."LENDING_PILOT" lp
join cashins a on lp.business_id = a.business_id
left join first_squareoff fs on a.product_id = fs.product_id
left join second_squareoff ss on a.product_id = ss.product_id
left join third_squareoff ts on a.product_id = ts.product_id

left join prod_db.data.transactions f
on a.first_cashin_trans_id = f.transaction_id
left join prod_db.data.transactions s 
on a.second_cashin_trans_id = s.transaction_id
left join prod_db.data.transactions t 
on a.third_cashin_trans_id = t.transaction_id

where third_cashin_date is not null
"""
)

In [4]:
pilot_df.shape

(643, 11)

In [5]:
pilot_df.head()

,product_id,business_id,pilot_first_due_date,pilot_second_due_date,pilot_third_due_date,pilot_fpd,pilot_spd,pilot_tpd,pilot_first_cure_days,pilot_second_cure_days,pilot_third_cure_days
0,2e7ebed2-1b44-4782-b567-cfbf4317a487,dd7f8b41-4ff5-4011-88ea-e244561bd9d3,2022-10-01 07:01:13.250000+00:00,2022-10-31 07:00:21.799000+00:00,2022-12-01 08:01:29.875000+00:00,1,1,1,2,1,1
1,1f52805d-dafb-4c32-8596-b94fa76e10a7,8c0c5bfb-95b0-4e1d-838d-66448a08364b,2022-08-04 07:00:00.122000+00:00,2022-08-07 07:00:01.160000+00:00,2022-08-11 07:00:00.439000+00:00,1,1,1,0,9,5
2,c6e016d4-9f44-42f1-af5d-c5d74f227400,66098cc5-63d3-4ff0-92db-69fb8a61c73b,2022-11-09 08:00:29.822000+00:00,2022-12-01 18:11:56.552000+00:00,2022-12-01 18:11:58.453000+00:00,1,0,1,0,0,0
3,8ab35049-2579-4b13-9f24-d7de3e2b2d3a,ace6c5d9-e919-4863-8693-c66255201f55,2022-10-30 07:00:51.525000+00:00,2022-11-30 08:00:58.393000+00:00,2022-12-30 08:00:18.329000+00:00,0,0,0,0,0,0
4,91775784-0aaf-4c93-9565-efb738bb716d,8ae481b2-d99a-4c51-9f95-2abc0c7b2a44,2022-09-26 07:00:30.807000+00:00,2022-10-28 08:20:06.372000+00:00,2022-11-26 08:08:00.041000+00:00,0,0,0,0,0,0


In [6]:
pilot_df = pilot_df[~pilot_df['pilot_third_due_date'].isnull()]

In [7]:
pilot_df.shape

(643, 11)

In [8]:
pilot_df.columns

Index(['product_id', 'business_id', 'pilot_first_due_date',
       'pilot_second_due_date', 'pilot_third_due_date', 'pilot_fpd',
       'pilot_spd', 'pilot_tpd', 'pilot_first_cure_days',
       'pilot_second_cure_days', 'pilot_third_cure_days'],
      dtype='object')

In [9]:
pilot_df['FPD_plus_3'] = 0
pilot_df['SPD_plus_3'] = 0
pilot_df['TPD_plus_3'] = 0
pilot_df.loc[(pilot_df['pilot_fpd']==1) & (pilot_df['pilot_first_cure_days']>=3),'FPD_plus_3'] = 1
pilot_df.loc[(pilot_df['pilot_spd']==1) & (pilot_df['pilot_second_cure_days']>=3),'SPD_plus_3'] = 1
pilot_df.loc[(pilot_df['pilot_tpd']==1) & (pilot_df['pilot_third_cure_days']>=3),'TPD_plus_3'] = 1

In [10]:
pilot_df['FPD_plus_15'] = 0
pilot_df['SPD_plus_15'] = 0
pilot_df['TPD_plus_15'] = 0
pilot_df.loc[(pilot_df['pilot_fpd']==1) & (pilot_df['pilot_first_cure_days']>=15),'FPD_plus_15'] = 1
pilot_df.loc[(pilot_df['pilot_spd']==1) & (pilot_df['pilot_second_cure_days']>=15),'SPD_plus_15'] = 1
pilot_df.loc[(pilot_df['pilot_tpd']==1) & (pilot_df['pilot_third_cure_days']>=15),'TPD_plus_15'] = 1

In [11]:
pilot_df['FPD_plus_30'] = 0
pilot_df['SPD_plus_30'] = 0
pilot_df['TPD_plus_30'] = 0
pilot_df.loc[(pilot_df['pilot_fpd']==1) & (pilot_df['pilot_first_cure_days']>=30),'FPD_plus_30'] = 1
pilot_df.loc[(pilot_df['pilot_spd']==1) & (pilot_df['pilot_second_cure_days']>=30),'SPD_plus_30'] = 1
pilot_df.loc[(pilot_df['pilot_tpd']==1) & (pilot_df['pilot_third_cure_days']>=30),'TPD_plus_30'] = 1

In [12]:
pilot_df.columns

Index(['product_id', 'business_id', 'pilot_first_due_date',
       'pilot_second_due_date', 'pilot_third_due_date', 'pilot_fpd',
       'pilot_spd', 'pilot_tpd', 'pilot_first_cure_days',
       'pilot_second_cure_days', 'pilot_third_cure_days', 'FPD_plus_3',
       'SPD_plus_3', 'TPD_plus_3', 'FPD_plus_15', 'SPD_plus_15', 'TPD_plus_15',
       'FPD_plus_30', 'SPD_plus_30', 'TPD_plus_30'],
      dtype='object')

In [13]:
querySno = SQLQuery('snowflake')
ga_df = querySno(r""" 
with default_flags as (
select distinct lending_business_id, business_id,loan_month,
max(case when emi_no = 1 then due_date end) over(partition by lending_business_id) as first_duedate, 
max(case when emi_no = 2 then due_date end) over(partition by lending_business_id) as second_duedate, 
max(case when emi_no = 3 then due_date end) over(partition by lending_business_id) as third_duedate,
max(case when emi_no = 1 then defalt end) over(partition by lending_business_id) as fpd, 
max(case when emi_no = 2 then defalt end) over(partition by lending_business_id) as spd, 
max(case when emi_no = 3 then defalt end) over(partition by lending_business_id) as tpd,
max(case when emi_no = 1 then total_emi end) over(partition by lending_business_id) as first_emi_amount,
max(case when emi_no = 2 then total_emi end) over(partition by lending_business_id) as second_emi_amount,
max(case when emi_no = 3 then total_emi end) over(partition by lending_business_id) as third_emi_amount
from (
select a.*,json_extract_path_text(c.predict_meta, 'business_id') as business_id, month(a.created_at) as month_no,
case when paid_on_time = 'true' then 0
when paid_on_time = 'false' then 1 else null end as default_flag,
dense_rank() over(partition by a.lending_business_id order by due_date) as emi_no,
min(a.created_at) over (partition by a.lending_business_id) as min_created_at,
month(min_created_at) as loan_month,
max(default_flag) over(partition by a.lending_business_id, due_date) as defalt,
sum(due_amount) over(partition by a.lending_business_id, due_date) as amount_due,
sum(payment) over(partition by a.lending_business_id, due_date) as payment_done,
amount_due/100 + payment_done/100 as total_emi
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS c on a.lending_business_id = c.lending_business_id
where date(due_date) < date('2023-04-04')
-- and a.lending_business_id = '909564ef-6b22-4303-9df5-fc4e892fef01'
)
)
, first_cure as (
select *,transaction_date as first_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) rnk
from (
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as first_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where first_amount_cured >= first_emi_amount
)
where rnk = 1
)
, second_cure as (
select *, transaction_date as second_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
t.running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as second_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where second_amount_cured >= first_emi_amount + second_emi_amount
)
where rnk = 1
)

,third_cure as (
select *, transaction_date as third_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as third_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where third_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount
)
where rnk = 1
)

select a.lending_business_id, a.business_id, a.first_duedate,date(f.first_cure_date)as first_cure_date, a.second_duedate,date(s.second_cure_date)as second_cure_date, 
a.third_duedate,date(t.third_cure_date) as third_cure_date, fpd, spd, tpd,
case when fpd = 1 and first_amount_cured >= a.first_emi_amount then 1 else 0 end as first_cure_flag,
case when spd = 1 and second_amount_cured >= a.first_emi_amount + a.second_emi_amount then 1 else 0 end as second_cure_flag,
case when tpd = 1 and third_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount then 1 else 0 end as third_cure_flag,
case when first_cure_flag = 1 then datediff(day,a.first_duedate,f.first_cure_date) else null end as first_cure_days,
case when second_cure_flag = 1 then datediff(day,a.second_duedate,s.second_cure_date) else null end as second_cure_days,
case when third_cure_flag = 1 then datediff(day,a.third_duedate,t.third_cure_date) else null end as third_cure_days,
a.first_emi_amount, f.first_amount_cured, a.second_emi_amount,(s.second_amount_cured - f.first_amount_cured) as second_amt_cured, a.third_emi_amount, 
(t.third_amount_cured - second_amt_cured) as third_amt_cured
from default_flags a 
left join first_cure f on a.lending_business_id = f.lending_business_id
left join second_cure s on a.lending_business_id = s.lending_business_id
left join third_cure t on a.lending_business_id = t.lending_business_id
where a.first_duedate < date('2023-04-19')
-- and a.business_id = '111eb19b-cd06-4943-b9f5-c111ae38189e'
order by first_duedate desc
""")

In [14]:
ga_df.shape

(1667, 23)

In [15]:
ga_df.sample(5)

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fpd,spd,tpd,first_cure_flag,second_cure_flag,third_cure_flag,first_cure_days,second_cure_days,third_cure_days,first_emi_amount,first_amount_cured,second_emi_amount,second_amt_cured,third_emi_amount,third_amt_cured
863,23b13461-a98e-4251-a6e9-0a44f498494e,e0ec23ed-d720-4ad4-8183-bbc7c09e199f,2023-02-27,2023-02-27,2023-03-27,2023-03-27,None,None,0,0.0,NaN,0,0,0,NaN,NaN,NaN,1820.83,1820.83,1820.83,1820.83,NaN,NaN
970,e71bf896-dc1a-42ce-9d42-2ece6d00d0ad,aaa76b70-e929-4d97-9e48-e90778384ec8,2023-02-25,2023-03-08,2023-03-25,2023-03-28,None,None,1,1.0,NaN,1,1,0,11.0,3.0,NaN,1022.66,1057.66,1589.28,1869.46,NaN,NaN
769,80534a0e-bec1-42e6-a247-6b7f8806a80e,90009ab3-dc99-41c8-8315-19c066ca3c89,2023-02-28,2023-02-28,2023-03-28,2023-03-28,None,None,0,0.0,NaN,0,0,0,NaN,NaN,NaN,322.66,322.66,322.66,322.66,NaN,NaN
233,18e378a0-0a3a-44e7-adc0-079826725abd,9565b4f4-a773-4fed-a52a-41d0c3bfa131,2023-03-22,None,None,None,None,None,0,NaN,NaN,0,0,0,NaN,NaN,NaN,436.00,NaN,NaN,NaN,NaN,NaN
1580,bbbe9426-6e6d-45d5-ad7e-2e61734f4898,aac7911b-7b36-41f5-a0a8-59b7572bc237,2023-01-01,2023-01-01,2023-02-01,2023-02-01,2023-03-04,2023-03-04,0,0.0,0.0,0,0,0,NaN,NaN,NaN,2201.33,2201.33,2688.16,2688.16,3132.66,5333.99


In [16]:
ga_df['FPD_plus_15'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=15),'FPD_plus_15'] = 1

In [17]:
#FPD_plus_15
ga_df['FPD_plus_15'].sum()/len(ga_df)

0.04919016196760648

In [18]:
ga_df['DPD_plus_15'] = 0
ga_df.loc[((ga_df['fpd']==1) & (ga_df['first_cure_days']>=15))| ((ga_df['spd']==1) & (ga_df['second_cure_days']>=15)) | ((ga_df['tpd']==1) & (ga_df['third_cure_days']>=15)),'DPD_plus_15'] = 1

In [19]:
#DPD_plus_15
ga_df['DPD_plus_15'].sum()/len(ga_df)

0.10737852429514097

In [20]:
ga_df['FPD_plus_30'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=30),'FPD_plus_30'] = 1

In [21]:
#FPD_plus_30
ga_df['FPD_plus_30'].sum()/len(ga_df)

0.02519496100779844

In [22]:
ga_df['SPD_plus_30'] = 0
ga_df['SPD_plus_15'] = 0
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=30),'SPD_plus_30'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=15),'SPD_plus_15'] = 1

In [23]:
ga_df['TPD_plus_30'] = 0
ga_df['TPD_plus_15'] = 0
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=30),'TPD_plus_30'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=15),'TPD_plus_15'] = 1

In [24]:
ga_df['FPD_plus_3'] = 0
ga_df['SPD_plus_3'] = 0
ga_df['TPD_plus_3'] = 0
ga_df.loc[(ga_df['fpd']==1) & (ga_df['first_cure_days']>=3),'FPD_plus_3'] = 1
ga_df.loc[(ga_df['spd']==1) & (ga_df['second_cure_days']>=3),'SPD_plus_3'] = 1
ga_df.loc[(ga_df['tpd']==1) & (ga_df['third_cure_days']>=3),'TPD_plus_3'] = 1

In [25]:
ga_df['DPD_plus_30'] = 0
ga_df.loc[((ga_df['fpd']==1) & (ga_df['first_cure_days']>=30))| ((ga_df['spd']==1) & (ga_df['second_cure_days']>=30)) | ((ga_df['tpd']==1) & (ga_df['third_cure_days']>=30)),'DPD_plus_30'] = 1

In [26]:
#DPD_plus_30
ga_df['DPD_plus_30'].sum()/len(ga_df)

0.07018596280743851

In [27]:
ga_df.columns

Index(['lending_business_id', 'business_id', 'first_duedate',
       'first_cure_date', 'second_duedate', 'second_cure_date',
       'third_duedate', 'third_cure_date', 'fpd', 'spd', 'tpd',
       'first_cure_flag', 'second_cure_flag', 'third_cure_flag',
       'first_cure_days', 'second_cure_days', 'third_cure_days',
       'first_emi_amount', 'first_amount_cured', 'second_emi_amount',
       'second_amt_cured', 'third_emi_amount', 'third_amt_cured',
       'FPD_plus_15', 'DPD_plus_15', 'FPD_plus_30', 'SPD_plus_30',
       'SPD_plus_15', 'TPD_plus_30', 'TPD_plus_15', 'FPD_plus_3', 'SPD_plus_3',
       'TPD_plus_3', 'DPD_plus_30'],
      dtype='object')

#### Identify Pilot migrators

In [28]:
pilot_mg = ga_df.merge(pilot_df,how='inner', left_on = 'business_id', right_on = 'business_id')

In [29]:
pilot_mg.shape

(331, 53)

#### GA STACKING

In [30]:
ga_first = ga_df[['business_id','first_duedate','FPD_plus_3','FPD_plus_15','FPD_plus_30']].rename(columns={'first_duedate': 'due_date', 'FPD_plus_3': 'DPD_plus_3','FPD_plus_15': 'DPD_plus_15','FPD_plus_30': 'DPD_plus_30'})
ga_second = ga_df[['business_id','second_duedate','SPD_plus_3','SPD_plus_15','SPD_plus_30']].rename(columns={'second_duedate': 'due_date', 'SPD_plus_3': 'DPD_plus_3','SPD_plus_15': 'DPD_plus_15','SPD_plus_30': 'DPD_plus_30'})
ga_third = ga_df[['business_id','third_duedate','TPD_plus_3','TPD_plus_15','TPD_plus_30']].rename(columns={'third_duedate': 'due_date', 'TPD_plus_3': 'DPD_plus_3','TPD_plus_15': 'DPD_plus_15','TPD_plus_30': 'DPD_plus_30'})

In [31]:
ga_first['emi_no'] = 1
ga_second['emi_no'] = 2
ga_third['emi_no'] = 3

In [32]:
ga_stack = pd.concat([ga_first, ga_second, ga_third], axis = 0, ignore_index = True)

In [33]:
ga_first.shape

(1667, 6)

In [34]:
ga_third.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
276,a7e6ab3c-5f73-4439-9a94-c67296bb4b1e,None,0,0,0,3
1371,0c83b0e9-f251-4a57-969b-2880b227aa03,2023-03-22,0,0,0,3
1144,e9b5cd85-7a04-4ef1-a327-dffa7f9b9765,None,0,0,0,3
30,298e53b5-58c7-42f1-9a4f-e017bca86460,None,0,0,0,3
486,fcaf1c99-f15e-44cb-88b7-ba3ad4256077,None,0,0,0,3


In [35]:
ga_stack.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
665,31a72693-4901-487f-bd29-2b87336adb21,2023-03-03,0,0,0,1
4906,c095b9d2-1a56-437c-bd79-e048ff26fcb7,2023-03-04,0,0,0,3
3105,21117a18-d563-428d-bff0-f0d263711822,2023-02-09,0,0,0,2
1170,dc104b9f-340c-4d01-bf4f-91b954be10ce,2023-02-16,0,0,0,1
282,ec808f21-f944-497c-b9fa-3ad121137f28,2023-03-19,0,0,0,1


In [36]:
ga_stack[ga_stack['due_date'].isnull()].shape

(1913, 6)

In [37]:
ga_stack.shape

(5001, 6)

In [38]:
ga_stack = ga_stack[~ga_stack['due_date'].isnull()]

In [39]:
ga_stack.shape

(3088, 6)

#### Pilot Stacking

In [40]:
pilot_df.head()

,product_id,business_id,pilot_first_due_date,pilot_second_due_date,pilot_third_due_date,pilot_fpd,pilot_spd,pilot_tpd,pilot_first_cure_days,pilot_second_cure_days,pilot_third_cure_days,FPD_plus_3,SPD_plus_3,TPD_plus_3,FPD_plus_15,SPD_plus_15,TPD_plus_15,FPD_plus_30,SPD_plus_30,TPD_plus_30
0,23c0016b-6b08-4bed-b4d4-95ba5b9eb659,763f6686-35cb-4578-bf6d-22219d8bcb0e,2022-09-30 07:00:25.667000+00:00,2022-10-30 07:00:43.963000+00:00,2022-11-30 08:00:51.547000+00:00,0,0,1,0,0,60,0,0,1,0,0,1,0,0,1
1,9b57a45c-754a-4300-ad60-2c4542fef5a5,7922550b-b6a4-47ca-8b53-e56a2c3e49d4,2022-07-05 07:00:00.147000+00:00,2022-08-05 07:00:00.306000+00:00,2022-08-27 07:00:31.631000+00:00,1,0,0,3,0,0,1,0,0,0,0,0,0,0,0
2,c9391332-013e-4528-9279-2aa561dc6de3,db972854-865b-424a-b264-bdc1c1ac058d,2022-10-02 07:00:40.985000+00:00,2022-11-01 07:00:48.751000+00:00,2022-11-30 18:14:34.996000+00:00,1,1,0,5,22,0,1,1,0,0,1,0,0,0,0
3,b2c1aeee-166b-4899-a297-20a195313ef4,26be705c-f5da-4ffa-8720-9b40dcdcee44,2022-08-17 07:00:00.406000+00:00,2022-09-17 07:00:00.090000+00:00,2022-10-17 07:00:00.084000+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,85a98d0f-5fb3-4dcd-9e47-435cf8555784,ab59e1a9-605b-43b4-8312-880b4123bc28,2022-10-01 07:00:44.111000+00:00,2022-10-15 07:06:47.568000+00:00,2022-10-31 07:00:16.200000+00:00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
pilot_df['pilot_first_due_date'] = pd.to_datetime(pilot_df['pilot_first_due_date']).dt.date
pilot_df['pilot_second_due_date'] = pd.to_datetime(pilot_df['pilot_second_due_date']).dt.date
pilot_df['pilot_third_due_date'] = pd.to_datetime(pilot_df['pilot_third_due_date']).dt.date

In [42]:
pilot_df.columns

Index(['product_id', 'business_id', 'pilot_first_due_date',
       'pilot_second_due_date', 'pilot_third_due_date', 'pilot_fpd',
       'pilot_spd', 'pilot_tpd', 'pilot_first_cure_days',
       'pilot_second_cure_days', 'pilot_third_cure_days', 'FPD_plus_3',
       'SPD_plus_3', 'TPD_plus_3', 'FPD_plus_15', 'SPD_plus_15', 'TPD_plus_15',
       'FPD_plus_30', 'SPD_plus_30', 'TPD_plus_30'],
      dtype='object')

In [43]:
first = pilot_df[['business_id','pilot_first_due_date','FPD_plus_3','FPD_plus_15','FPD_plus_30']].rename(columns={'pilot_first_due_date': 'due_date', 'FPD_plus_3': 'DPD_plus_3','FPD_plus_15':'DPD_plus_15','FPD_plus_30':'DPD_plus_30'})
second = pilot_df[['business_id','pilot_second_due_date','SPD_plus_3','SPD_plus_15','SPD_plus_30']].rename(columns={'pilot_second_due_date': 'due_date', 'SPD_plus_3': 'DPD_plus_3','SPD_plus_15':'DPD_plus_15','SPD_plus_30':'DPD_plus_30'})
third = pilot_df[['business_id','pilot_third_due_date','TPD_plus_3','TPD_plus_15','TPD_plus_30']].rename(columns={'pilot_third_due_date': 'due_date', 'TPD_plus_3': 'DPD_plus_3','TPD_plus_15':'DPD_plus_15','TPD_plus_30':'DPD_plus_30'})

In [44]:
first['emi_no'] = 1
second['emi_no'] = 2
third['emi_no'] = 3

In [45]:
third.shape

(643, 6)

In [46]:
third.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
453,fdd8a93d-a5b6-46ba-a461-aff279e05f17,2022-10-20,1,1,1,3
333,206dce8f-1527-48ca-b9aa-11fe81ed6c9c,2022-11-30,0,0,0,3
361,bf8ec046-4e61-4acc-bdf1-992c752d50d3,2022-10-18,0,0,0,3
202,545b8198-4033-489f-a84f-fc02d8ad19b0,2022-11-08,1,1,1,3
362,b19b13fd-4860-482c-9588-2b90205decad,2022-09-05,0,0,0,3


In [47]:
pilot_stack = pd.concat([first, second, third], axis = 0, ignore_index = True)

In [48]:
pilot_stack.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
1568,3d4b951b-84f7-4a04-9614-53b6b1d4a4e5,2022-11-22,0,0,0,3
1253,100e66f2-6836-4801-ab35-c43d3b01840a,2022-10-01,0,0,0,2
623,a42abe38-52fc-4dbf-9fde-cda5ef443787,2022-08-22,1,1,0,1
1394,3608a3f0-5a73-4682-80ca-e680aad3c4ad,2022-11-21,1,1,0,3
419,d65c3d0f-9eb3-4a2f-bc33-317b86801ef0,2022-08-26,0,0,0,1


In [49]:
pilot_stack.shape

(1929, 6)

#### Adjusting EMI_NO for pilot migrated accounts

In [50]:

ga_stack.loc[(ga_stack['business_id'].isin(list(pilot_mg['business_id']))), 'emi_no'] = ga_stack.loc[(ga_stack['business_id'].isin(list(pilot_mg['business_id']))), 'emi_no'] + 3

#### Merging Pilot and GA Stacked

In [51]:
dff = pd.concat([ga_stack, pilot_stack], axis = 0, ignore_index = True)

In [52]:
dff.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
327,d50567c6-5050-4747-bf1d-036a62ec42af,2023-03-16,0,0,0,1
4931,8929c03f-4958-46b4-8b6b-70ec12895c67,2022-11-12,0,0,0,3
3509,c0c82e1d-9e25-4a01-ba0a-99b8a62c28a0,2022-07-26,0,0,0,1
1587,b51ef33b-8e92-458b-98c7-b19408b5beff,2023-01-01,0,0,0,4
4400,7f6308a5-c1bd-4d67-afff-4ca1de634846,2023-01-18,0,0,0,3


In [53]:
dff.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
947,21678f69-7202-498f-a79a-c708eb5abeae,2023-02-25,0,0,0,1
4412,edb4bb54-56e9-4849-9975-2f6a5db9c4e7,2022-09-22,0,0,0,3
4440,d60b8e98-ebbb-4e9b-80c8-02a3e3b58e9f,2022-11-09,0,0,0,3
3038,162472a9-a6ae-4ea8-89b9-279ec33dca3b,2023-03-03,1,1,1,6
3218,9af75dbc-53a8-4e52-96bc-aa5fb15a0668,2022-09-11,0,0,0,1


In [54]:
dff.shape

(5017, 6)

In [55]:
# dff[dff['emi_no']==6].head()

In [56]:
dff[dff['business_id']== '3ba4a270-25c2-4160-84b7-0a7b6e6bf618'].sort_values('emi_no')

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
917,3ba4a270-25c2-4160-84b7-0a7b6e6bf618,2023-02-26,0,0,0,1
1992,3ba4a270-25c2-4160-84b7-0a7b6e6bf618,2023-03-26,0,0,0,2


In [57]:
dff[dff['business_id']== 'ddc3a22d-3205-4d6e-900a-201605bbfa9c'].sort_values('emi_no')

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
3536,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2022-10-20,0,0,0,1
4179,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2022-11-20,0,0,0,2
4822,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2022-12-20,1,0,0,3
1332,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2023-02-03,1,1,1,4
2407,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2023-03-06,0,0,0,5
2753,ddc3a22d-3205-4d6e-900a-201605bbfa9c,2023-04-03,0,0,0,6


In [58]:
dff['emi_no'].value_counts()

1    1979
2    1397
3     706
4     331
5     321
6     283
Name: emi_no, dtype: int64

In [59]:
dff['DPD_plus_3'].sum()/len(dff)

0.14351205899940203

In [60]:
dff[dff['emi_no']==1]

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
0,e2a8d148-50d0-4d89-b515-cc29815b03ff,2023-04-03,0,0,0,1
1,e6a7d2a0-2583-446a-8c25-889077271ef0,2023-04-03,0,0,0,1
2,d3314f96-5e59-4e28-b7bc-0cc6332691d0,2023-04-03,0,0,0,1
3,e6e256db-28d2-42f3-9962-63afc6897641,2023-04-03,0,0,0,1
4,e26e86ab-42ab-422c-8ee6-bd57eb20032d,2023-04-03,0,0,0,1
...,...,...,...,...,...,...
3726,fd09139f-a3e5-4f10-bc85-07c9037088ad,2022-10-11,0,0,0,1
3727,8dad6e58-4c7d-42b0-87f6-0ab0eef2306d,2022-09-29,0,0,0,1
3728,1872a8e8-ed78-4387-8d9a-99bc0e848790,2022-10-10,0,0,0,1
3729,2223f662-f3ad-42e7-b5a6-4a62221dd726,2022-08-28,0,0,0,1


In [61]:
# querySno = SQLQuery('snowflake')
# ga_flat = querySno(r""" 
# select distinct business_id, emi_no, due_date, defalt from (
# select a.*,json_extract_path_text(c.predict_meta, 'business_id') as business_id, month(a.created_at) as month_no,
# case when paid_on_time = 'true' then 0
# when paid_on_time = 'false' then 1 else null end as default_flag,
# dense_rank() over(partition by a.lending_business_id order by due_date) as emi_no,
# min(a.created_at) over (partition by a.lending_business_id) as min_created_at,
# month(min_created_at) as loan_month,
# max(default_flag) over(partition by a.lending_business_id, due_date) as defalt,
# sum(due_amount) over(partition by a.lending_business_id, due_date) as amount_due,
# sum(payment) over(partition by a.lending_business_id, due_date) as payment_done,
# amount_due/100 + payment_done/100 as total_emi
# from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a
# join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS c on a.lending_business_id = c.lending_business_id
# where date(due_date) < dateadd(month,2,date('2023-04-19'))
# -- and a.lending_business_id = '909564ef-6b22-4303-9df5-fc4e892fef01'
# )
# where defalt is not null
# order by 1,2,3


# """)

In [62]:
# from sqlalchemy.types import NVARCHAR
# # from conf.config import SQLQuery
# q = SQLQuery('snowflake')

# dff.to_sql(name='lending_dpd_data_pilot_ga',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in dff})